<a href="https://colab.research.google.com/github/rrsartneoai/123coder/blob/main/testowy_21_dobry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers datasets evaluate pyarrow==14.0.1

from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments
from evaluate import evaluator
from datasets import load_dataset, load_metric
import numpy as np # Import numpy for argmax

# Załaduj dane z podziałem na treningowy i walidacyjny
# Assign the splits to separate variables
train_dataset, eval_dataset = load_dataset('text', data_files={'train': '/content/sample_data/train.txt'}, split=['train[:80%]', 'train[80%:]'] )

# Inicjalizacja tokenizer'a i modelu
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Funkcja do przypisania etykiet (popraw na podstawie swoich danych)
def add_labels(example):
    if len(example['text']) > 10:
        example['labels'] = 1
    else:
        example['labels'] = 0
    return example

# Apply the add_labels function to both training and validation splits
train_dataset = train_dataset.map(add_labels)
eval_dataset = eval_dataset.map(add_labels)

# Tokenizacja danych
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, labels=examples['labels'])

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)

# Parametry fine-tuningu
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=1e-5,
    eval_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,  # Loguj stratę co 10 kroków
    evaluation_strategy="epoch" # Dodane - Rafał - aby wyświetlało training loss
)

# Metryka
metric = load_metric("accuracy")

# Funkcja do obliczania metryk
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Inicjalizacja i uruchomienie fine-tuningu
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset, # Use the tokenized training dataset
    eval_dataset=tokenized_eval_dataset, # Use the tokenized validation dataset
    compute_metrics=compute_metrics,
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the n

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.354895,0.928571
2,0.393800,0.308780,0.928571
